# НАЧАЛО РАБОТЫ

## Импорт библиотек

In [ ]:
#!pip install category_encoders
#!pip freeze > requirements.txt
#!pip install <pkg>
#!pip install datawig --user 
#!pip install datawig 
#!pip install pandas-profiling

In [ ]:
#загрузка необходимых библиотек
import pandas as pd
#import datawig
import numpy as np
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn import preprocessing
import category_encoders as ce # импорт для работы с кодировщиком
import pickle
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer
#    nltk используем для оценки текста комментария
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from pandas_profiling import ProfileReport

## Настройка, подгрузка датасета

In [ ]:
# зафиксировал random seed и версию пакетов, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42
!pip freeze > requirements.txt

# Подгрузим наши данные из соревнования
DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') 
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') 
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') 

# для корректной обработки признаков объединяем трейн и тест в один датасет (позже разделим)
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями
data = df_test.append(df_train, sort=False).reset_index(drop=True) 

In [ ]:
print(df_test.shape)
print(data.head())
print(data.info())

# Посмотрим на профайлер


In [ ]:
profile = ProfileReport(data, title="Booking_hotels")
profile

* Можно сделать следующие выводы:515738 строки,336 дубликатов(удалить), сильно коррелируют признаки total_number_of_rewies/additional_number_of_scoring (можно удалить один из них), есть пропущенные значения в признаках lat и lng (заполнить,подумать чем)

# ОБРАБОТКА ДАТАСЕТА

# 1. Удалим дубликаты 
*(из-за удаления дубликатов тест не совпадает с сабмитом, поэтому я отказалась от этой идеи)

In [ ]:
#data = data.drop_duplicates(ignore_index=True)

# 2. Удалим признак additional_number_of_scoring
т.к. этот признак высоко коррелирует c total_number_of_rewies

In [ ]:
data = data.drop('additional_number_of_scoring',axis = 1)
hotels = data.copy()

# 3. Заполним пустые значения широты и долготы
*применим методы машинного обучения (импутация с помощью datawig)

In [ ]:
hotels['lat'] = hotels['lat'].fillna(np.nan)
hotels['lng'] = hotels['lng'].fillna(np.nan)
hotels_test = hotels[pd.isnull(hotels['lat'])|pd.isnull(hotels['lng'])]
hotels_train = hotels[pd.notnull(hotels['lat'])|pd.notnull(hotels['lng'])]
# Инициализируем модель SimpleImputer
imputer = datawig.SimpleImputer(
  input_columns=['lat','hotel_address'], # из каких столбцов брать информацию для импутации
  output_column= 'lng', # какой столбец восстанавливаем
  output_path = 'imputer_model_2' # куда записывать модель и её метрики
  )
# Тренируем модель
imputer.fit(train_df=hotels_train, num_epochs=50)
# Проводим импутацию и возвращаем восстановленный набор данных
imputed = imputer.predict(hotels_test)

In [ ]:
imputed.to_pickle('hotels_lat_imputed.csv')   # сохранения применяла  при работе на локальном диске, сохраняю тут код(для себя)
lat = pd.read_pickle(r'hotels_lat_imputed.csv')
hotels_test['lat'] = lat['lat_imputed']


In [ ]:
imputed.to_pickle('hotels_lng_imputed.csv')# сохранения применяла  при работе на локальном диске, сохраняю тут код(для себя)
lng = pd.read_pickle(r'hotels_lng_imputed.csv')
hotels_test['lng'] = lng['lng_imputed']

In [ ]:
data = pd.concat([hotels_train,hotels_test],axis = 0).reset_index() 
data = data.drop('lat_imputed',axis = 1)
data = data.drop('lng_Imputed',axis = 1)

In [ ]:
data = data.drop('index', axis = 1)
data.info()


In [ ]:
data.to_scv('data.csv') # сохранение в среде kaggle


In [ ]:
# Загрузка данных с заполненными пропусками
data = pd.read_csv('data.csv')

# Проверим еще раз коррреляцию

In [ ]:
sns.heatmap(data.corr())

*Сильной корреляции между признаками не наблюдается

# удалим признак days_since_review
*ведь мы не решаем задачу предсказания оценки в рамках временного ряда, видится логичным не оценивать этот параметр

In [ ]:
data = data.drop('days_since_review',axis =1)

# 3. Обработка даты
*От даты написания отзыва оставлю только месяц, т.к. считаю,что именно месяц может как-то влиять на отзыв в долгосрочном плане( например,может прослеживаться разница между отзывами в ноябре и отзывами в июле)

In [ ]:
data['review_date'] = pd.to_datetime(data['review_date'])  
data['review_date_month'] = pd.DatetimeIndex(data['review_date']).month
data= data.drop('review_date',axis =1)
data.shape

In [ ]:
#обработаем месяцы OneHotEncoderom, создав 12 новых признаков в бинарном формате
encoder = ce.OneHotEncoder(cols=['review_date_month']) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data['review_date_month'])
data = pd.concat([data, type_bin], axis=1).reset_index()
data = data.drop('review_date_month',axis = 1)
data.shape

# 4. Стандартизируем числовые признаки

In [ ]:
col_name= data[['average_score','review_total_negative_word_counts','review_total_positive_word_counts','total_number_of_reviews_reviewer_has_given','total_number_of_reviews']]

In [ ]:
scaler = preprocessing.StandardScaler()
data_scalar = scaler.fit_transform(col_name)
data_scalar = pd.DataFrame(data_scalar,columns=col_name.columns)

In [ ]:
data_scalar.sample(3)

In [ ]:
#заменяю признаки на скалированные признаки в исходном датасете
data = data.drop(col_name,axis =1)
data_numeric = pd.concat([data,data_scalar],axis =1).reset_index()
data_numeric = data_numeric.drop('index',axis = 1)
data_numeric.info()

# Обработка текстовых признаков:
# 1. Разберемся с адресом отеля 

In [ ]:
#взглянем на содержание
data_numeric['hotel_address']

In [ ]:
#оставлю только страну( т.к. весь датасет собран всего из 5ти городов ,1 город в 1 стране)
data_numeric['hotel_town'] = data['hotel_address'].apply(lambda x: x.split(' ')[-1:]) 
data_numeric['hotel_town']= data_numeric['hotel_town'].astype('str')

In [ ]:
# 5 стран- обработаем OneHotEncoder,создавая 5 новых бинарных признаков
encoder = ce.OneHotEncoder(cols = ['hotel_town']) # указываем столбец для кодирования
type_bin = encoder.fit_transform(data_numeric['hotel_town'])
data_num= pd.concat([data_numeric, type_bin], axis=1)
data_num = data_num.drop(['hotel_town','hotel_address'],axis = 1)
data_num.shape


In [ ]:
data_num.sample(2)

 # 2. Разберемся с названием отеля

In [ ]:
# в датасете более 1000 уникалтных названий отелей, обработаем их LabelEncoder,
# а затем проскалируем значения,чтоб не преувеличивать вес высоких лейблов
data_num['hotel_name'] = LabelEncoder().fit_transform(data_num['hotel_name'])
temp_array = np.array(data_num['hotel_name']).reshape(data_num.shape[0],1)
data_num_scalar = scaler.fit_transform(temp_array)
data_num_text = pd.DataFrame(data_num_scalar,columns=['hotel_scalar_name'])
data_num_text.head(1)

In [ ]:
# заменим hotel_name на скалированное значение
data_num = data_num.drop('hotel_name',axis =1)
data_before_itog = pd.concat([data_num,data_num_text],axis =1)
data_before_itog.info()

In [ ]:
data_before_itog.sample(2)

# 3. Обработка тегов

In [ ]:
#Из всего набора тегов оставим 10 наиболее популярных в виде бинарных признаков
TAGS_NUMBER = 11
c = Counter()
for tags in data['tags'].apply(lambda s: [x.strip() for x in s[1:-1].replace("'",'').split(',')]):
    for tag in tags:
        c[tag] += 1
for tag , _ in c.most_common(TAGS_NUMBER):
    tag_name = tag.lower().replace(' ','_')
    data_before_itog[f"tag_{tag_name}"] = data_before_itog['tags'].apply(lambda tags: int(tag in tags))        
data_before_itog.drop(['tags','tag_leisure_trip'],axis = 1, inplace = True)

In [ ]:
data_before_itog = data_before_itog.drop('level_0',axis = 1)

In [ ]:
data_before_itog.head(1)

# 4. Национальности

In [ ]:
# Возьмем 10 самых частых национальностей,остальные пометик как other, обрабоаем labelEncoder
data_before_itog['reviewer_nationality'].nunique()
pop_suburb= data_before_itog['reviewer_nationality'].value_counts().nlargest(10).index
data_before_itog['reviewer_nationality'] = data_before_itog['reviewer_nationality'].apply(lambda x:x if x in pop_suburb else 'Other')
data_before_itog['reviewer_nationality'] = LabelEncoder().fit_transform(data_before_itog['reviewer_nationality'])
data_before_itog.head(1)

# 5. Обработка отзывов

In [ ]:
replace = {    
    'positive_review': ['no positive','nothing'],
    'negative_review': [
        'nothing really','no negative','nothing','n a','none','nothing at all','nothing to dislike',
        'everything was perfect','na','can t think of anything','nil','everything was great','absolutely nothing',
        'nothing to complain about','no','nothing not to like','nothing all good','no complaints','i liked everything'
        ,'liked everything'
    ]
    
}

In [ ]:
for key in replace:
    for value in replace[key]:
        data_before_itog.loc[data_num[key].str.strip().str.lower()==value,key] = ''
        
data_before_itog['review_total_positive_word_counts'] = data_before_itog['positive_review'].apply(lambda s: len(s.strip().split(' ')))
data_before_itog['review_total_negative_word_counts'] = data_before_itog['negative_review'].apply(lambda s: len(s.strip().split(' ')))

data_before_itog['review_diff'] = data_before_itog['positive_review'].str.len() - data_before_itog['negative_review'].str.len()

polarity_columns = ['neg','neu','pos','compound']

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
def get_polarity(row, analyzer):
    positive_counter = Counter(analyzer.polarity_scores(row['positive_review']))
    negative_counter = Counter(analyzer.polarity_scores(row['negative_review']))
    result_counter = positive_counter + negative_counter
    return [result_counter[col] for col in polarity_columns]

polarities = list(data_before_itog.apply(get_polarity, analyzer = analyzer, axis = 1))

data_polarity = pd.DataFrame(polarities,columns = ['neg','neu','pos','compound'])

In [ ]:
data_itog = pd.concat([data_before_itog, data_polarity], axis = 1)
data_itog.drop(['positive_review','negative_review'],axis = 1, inplace = True)
data_itog.sample(2)

# Создание и обучение модели

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

In [ ]:
data_itog = data_itog.drop(['lat','lng'],axis =1) # datawig установить не удалось,он приносил свои баллы к score на локальном диске, тут попробую совсем удалить эти признаки

## Проверка на мультиколлинеарность

In [ ]:
import plotly.express as px

In [ ]:
# корреляция признаков (оставляем только сильную связь для наглядности)
pivot = data_itog.corr()
pivot = pivot.drop(['sample', 'reviewer_score'], axis=0)
pivot = pivot.drop(['sample', 'reviewer_score'], axis=1)
for col in pivot:
    pivot[col] = pivot[col]   #.apply(lambda x: np.nan if (abs(x) < 0.85 or x == 1) else x)
for col in pivot:
    pivot = pivot.dropna(how='all')
    pivot = pivot.dropna(how='all', axis='columns')

fig = px.imshow(pivot)
fig.show()

In [ ]:
# постараемся удалить как можно меньше признаков. Пробуем убрать как можно меньше признаков,
# но признаки с максимальной корреляцией в приоритете для удаления
counter = 0
for lower_bound in np.linspace(0.98, 0.85, num=14):
    for col in pivot:
        if pivot[col].max() > lower_bound or pivot[col].min() < -lower_bound:
            pivot = pivot.drop(col, axis=0)
            pivot = pivot.drop(col, axis=1)
            data = data_itog.drop(col, axis=1)
            counter += 1
print('Deleted', counter, 'columns')

# ОБУЧЕНИЕ МОДЕЛИ

In [ ]:
# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

In [ ]:
# выделим трейн-тест-признаки-таргет
train_data = data_itog.query('sample == 1').drop(['sample'], axis=1)
test_data = data_itog.query('sample == 0').drop(['sample'], axis=1)
y = train_data.reviewer_score.values            
X = train_data.drop(['reviewer_score'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
print(y.shape,X.shape, X_train.shape, X_test.shape)

In [ ]:
test_data.shape

In [ ]:
# в качестве модели возьмем случайный лес
from sklearn.ensemble import RandomForestRegressor 
from sklearn import metrics # инструменты для оценки точности модели
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
# оцениваем точность прогноза
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print('MAPE:', mean_absolute_percentage_error(y_test, y_pred)) 

In [ ]:
import matplotlib as plt

In [ ]:
# выводим самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(50).plot(kind='barh')

In [ ]:
# готовим ответ
#test_data = test_data.drop(['reviewer_score'], axis=1)
predict_submission = model.predict(test_data)
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission2.csv', index=False)

In [ ]:
pd.read_csv('submission2.csv')